In [1]:
# Import required packages
import torch
from tqdm import tqdm

import wandb
# from utils.dataset import create_dataset
from utils.model import *
from utils.robot import Robot
from utils.settings import config as cfg
from utils.utils import *

robot = Robot(verbose=False)
J_tr, P_tr = data_collection(robot=robot, N=cfg.N_train)
_, P_ts = data_collection(robot=robot, N=cfg.N_test)
F = posture_feature_extraction(J_tr)
knn = get_knn(P_tr=P_tr)

hnne load successfully from /home/luca/ikpflow/weights/panda/hnne-4.pickle
knn load successfully from /home/luca/ikpflow/weights/panda/knn-3.pickle


In [ ]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [ ]:
panda = Robot(verbose=False)
# data generation
J_tr, P_tr = data_collection(robot=panda, N=config.N_train)
J_ts, P_ts = data_collection(robot=panda, N=config.N_test)
F = posture_feature_extraction(J_tr)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()
# knn = get_knn(P_tr)
knn = get_knn(P_tr=P_tr)

In [ ]:
mu = torch.zeros(size=(config.n,))
NUM_DATA = 100
NUM_SAMPLES = 1000

In [ ]:
test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=flow, knn=knn, K=NUM_SAMPLES, print_report=True)

In [ ]:
nflow = get_nflow_model(flow=flow)
inference(robot=panda, P_inf=P_ts[:NUM_DATA], F=F, solver=nflow, knn=knn, K=NUM_SAMPLES, print_report=True)

In [ ]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0.31)
test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=iflow, knn=knn, K=NUM_SAMPLES, print_report=True)

In [ ]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

In [ ]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 100
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

In [ ]:
def get_target_poses(P_ts):
    quaternions = np.zeros((len(P_ts), 4))
    quaternions[:, 0] = np.random.randn() * 2e-2 + 1
    target_poses = np.column_stack((P_ts, quaternions))
    return target_poses

In [ ]:
target_poses = get_target_poses(P_ts[:NUM_DATA])
target_poses.shape

In [ ]:
# -> unrefined solutions
l2_errs = np.zeros((len(target_poses), NUM_SAMPLES))
time_diffs = np.zeros((len(target_poses), NUM_SAMPLES))

for i, target_pose in enumerate(target_poses):
    iksols, l2_err, _, _, _, dt = ik_solver.solve(
        target_pose, 
        n=NUM_SAMPLES,
        refine_solutions=False, 
        return_detailed=True)

    iksols = iksols.detach().cpu().numpy()
    dt = np.zeros_like(l2_err) + dt
    
    l2_errs[i] = l2_err
    time_diffs[i] = dt

l2_errs = l2_errs.reshape((-1))
time_diffs = time_diffs.reshape((-1))

    
df = pd.DataFrame(
    data=np.column_stack((l2_errs, time_diffs)),
    columns=["l2_err", "time_diff"],
)

In [ ]:
df.describe()